# Discretiser - Journey Simulation Examples

This notebook demonstrates the discretiser framework for simulating patient journeys with competing events.

**Note:** These examples are illustrative of the framework's machinery rather than realistic clinical models. In particular:
- Survival parameters are arbitrary, not calibrated to real data
- Outcome/censoring do not depend on treatment history
- No neural network integration for state-dependent dynamics

The framework supports all of the above via custom `SurvivalModel` subclasses that access `state` features (event counts, times, embeddings) in their `sample()` method.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Add parent directory to path so discretiser package can be found
repo_root = Path.cwd().parent.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
from discretiser.src import (
    # Survival models
    Weibull, Exponential, PointMassPlusContinuous, Mixture, NeverOccurs,
    StateDependentWeibull,
    # Events
    EventType, EventRegistry, TriggerRule,
    make_censoring_event, make_terminal_event, make_triggering_event,
    # State
    Subject, State, generate_subjects,
    # Simulator
    Simulator, simulate_cohort_simple
)

## 1. Basic Example: Simple Patient Journey

A simple model with diagnosis, treatment, outcome, and censoring.

In [ ]:
# Define event types with realistic dependencies
# Diagnosis occurs once, but treatment can recur (e.g., multiple sessions)
diagnosis = EventType('diagnosis', Weibull(shape=1.5, scale=30))
diagnosis.active_condition = lambda state, _: 'diagnosis' not in state.occurred_events

# Treatment only possible after diagnosis, can recur
treatment = EventType('treatment', Weibull(shape=1.2, scale=14))
treatment.active_condition = lambda state, _: 'diagnosis' in state.occurred_events

events = [
    diagnosis,
    treatment,
    EventType('outcome', Weibull(shape=2.0, scale=180), terminal=True),
    make_censoring_event('censoring', Exponential(rate=1/365))
]

# Simulate a cohort
df = simulate_cohort_simple(n_subjects=200, events=events, max_time=365)
print(f"Total events: {len(df)}")
print(f"Events per subject: {df.groupby('subject_id').size().describe()}")
df['event'].value_counts()

In [ ]:
# Visualise event times
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Event counts
df['event'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Event Counts')
axes[0].set_ylabel('Count')

# Time distribution by event
for event in df['event'].unique():
    subset = df[df['event'] == event]
    axes[1].hist(subset['time'], bins=30, alpha=0.5, label=event)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Event Time Distributions')
axes[1].legend()

plt.tight_layout()
plt.show()

## 2. Triggered Events: Diagnosis Triggers Treatment

Using `PointMassPlusContinuous` to model simultaneous events:
- 30% chance treatment starts immediately at diagnosis
- 70% chance treatment follows a Weibull distribution

In [ ]:
# Diagnosis can trigger treatment (diagnosis occurs once)
diagnosis = make_triggering_event(
    name='diagnosis',
    survival_model=Weibull(1.5, 60),
    triggers={
        # 30% immediate, 70% follows Weibull (competing with other events)
        'treatment': PointMassPlusContinuous(p0=0.3, continuous=Weibull(1.2, 14))
    }
)
diagnosis.active_condition = lambda state, _: 'diagnosis' not in state.occurred_events

# Treatment can also recur independently after first occurrence
# (e.g., follow-up treatments after the triggered one)
treatment = EventType('treatment', Weibull(1.5, 30))
treatment.active_condition = lambda state, _: 'treatment' in state.occurred_events

# Note: To model "sometimes no treatment at all", use Mixture with NeverOccurs:
#   Mixture(
#       models=[NeverOccurs(), PointMassPlusContinuous(0.3, Weibull(1.2, 14))],
#       weights=[0.2, 0.8]  # 20% never triggered, 80% triggered
#   )

events_triggered = [
    diagnosis,
    treatment,
    EventType('outcome', Weibull(2.0, 180), terminal=True),
    make_censoring_event('censoring', Exponential(rate=1/365))
]

df_triggered = simulate_cohort_simple(n_subjects=200, events=events_triggered, max_time=365)
print("Events per subject:", df_triggered.groupby('subject_id').size().describe())
print("\nTriggered events breakdown:")
print(df_triggered['triggered_by'].value_counts(dropna=False))

In [ ]:
# Analyse delay from diagnosis to first treatment
# Use merge_asof to correctly match each treatment to the preceding diagnosis
treatments = df_triggered[df_triggered['event'] == 'treatment'].copy()
diagnoses = df_triggered[df_triggered['event'] == 'diagnosis'][['subject_id', 'time']].copy()
diagnoses = diagnoses.rename(columns={'time': 'diagnosis_time'})

# Sort by the merge key for merge_asof
treatments_sorted = treatments.sort_values('time')
diagnoses_sorted = diagnoses.sort_values('diagnosis_time')

# Match each treatment to the most recent diagnosis before it
merged = pd.merge_asof(
    treatments_sorted,
    diagnoses_sorted,
    left_on='time',
    right_on='diagnosis_time',
    by='subject_id',
    direction='backward'
)
merged['delay'] = merged['time'] - merged['diagnosis_time']

print(f"Total treatments: {len(merged)}")
print(f"Simultaneous (delay=0): {(merged['delay'] == 0).sum()}")
print(f"Delayed (delay>0): {(merged['delay'] > 0).sum()}")

# Plot delay distribution
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(merged['delay'].dropna(), bins=50, edgecolor='black')
ax.axvline(0, color='red', linestyle='--', label='Simultaneous (t=0)')
ax.set_xlabel('Delay from Diagnosis to Treatment')
ax.set_ylabel('Count')
ax.set_title('Treatment Delay Distribution (with Point Mass at 0)')
ax.legend()
plt.show()

## 3. Subject-Dependent Survival (Covariates)

Subject features can modify survival model parameters. Here we show how to create
subject-dependent Weibull parameters where the scale depends on subject features.

In [ ]:
class SubjectDependentWeibull(Weibull):
    """
    Weibull where scale depends on subject features.
    
    scale_adjusted = base_scale * exp(subject_features @ coefficients)
    """
    
    def __init__(self, shape: float, base_scale: float, coefficients: np.ndarray):
        super().__init__(shape, base_scale)
        self.base_scale = base_scale
        self.coefficients = coefficients
    
    def sample(self, state=None, subject=None) -> float:
        if subject is not None and subject.feature_vector is not None:
            # Adjust scale based on subject features
            log_modifier = np.dot(subject.feature_vector, self.coefficients)
            adjusted_scale = self.base_scale * np.exp(log_modifier)
        else:
            adjusted_scale = self.base_scale
        
        return adjusted_scale * np.random.weibull(self.shape)

# Feature dimension = 4, create coefficients
# Positive coefficients -> longer time to event
coef_diagnosis = np.array([0.5, -0.3, 0.2, -0.1])  # Feature effects on diagnosis timing
coef_outcome = np.array([-0.4, 0.3, -0.2, 0.5])    # Feature effects on outcome timing

print("Coefficients for diagnosis:", coef_diagnosis)
print("Coefficients for outcome:", coef_outcome)

In [ ]:
# Create events with subject-dependent survival
# Diagnosis once, treatment can recur
diagnosis_cov = EventType('diagnosis', SubjectDependentWeibull(1.5, 60, coef_diagnosis))
diagnosis_cov.active_condition = lambda state, _: 'diagnosis' not in state.occurred_events

treatment_cov = EventType('treatment', Weibull(1.2, 14))
treatment_cov.active_condition = lambda state, _: 'diagnosis' in state.occurred_events

events_covariate = [
    diagnosis_cov,
    treatment_cov,
    EventType('outcome', SubjectDependentWeibull(2.0, 180, coef_outcome), terminal=True),
    make_censoring_event('censoring', Exponential(rate=1/365))
]

# Generate subjects with different feature profiles
subjects = generate_subjects(n=200, feature_dim=4, dirichlet_alpha=0.5)

# Show feature distribution
features_df = pd.DataFrame(
    [s.feature_vector for s in subjects],
    columns=[f'f{i}' for i in range(4)]
)
print("Subject feature summary:")
print(features_df.describe())

In [ ]:
# Run simulation
registry = EventRegistry()
registry.register_all(events_covariate)

simulator = Simulator(registry, max_time=365)
results = simulator.simulate_cohort(subjects)

df_covariate = simulator.cohort_to_dataframe(results)
print(f"Total events: {len(df_covariate)}")
df_covariate['event'].value_counts()

In [ ]:
# Analyse relationship between subject features and event times
# Get first diagnosis time per subject
first_diagnosis = df_covariate[df_covariate['event'] == 'diagnosis'].groupby('subject_id')['time'].min()

# Merge with features
analysis_df = features_df.copy()
analysis_df['subject_id'] = [s.id for s in subjects]
analysis_df['diagnosis_time'] = analysis_df['subject_id'].map(first_diagnosis)

# Plot feature vs diagnosis time
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for i, ax in enumerate(axes.flat):
    col = f'f{i}'
    valid = analysis_df.dropna(subset=['diagnosis_time'])
    ax.scatter(valid[col], valid['diagnosis_time'], alpha=0.5)
    ax.set_xlabel(f'Feature {i}')
    ax.set_ylabel('Diagnosis Time')
    ax.set_title(f'Coef: {coef_diagnosis[i]:+.2f}')

plt.suptitle('Subject Features vs Diagnosis Time')
plt.tight_layout()
plt.show()

## 4. Visualising Individual Journeys

In [ ]:
def plot_journeys(df, n_subjects=10):
    """Plot event timelines for a sample of subjects."""
    subjects = df['subject_id'].unique()[:n_subjects]
    
    # Assign colours to events
    event_types = df['event'].unique()
    colours = dict(zip(event_types, plt.cm.Set2.colors[:len(event_types)]))
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for i, subj in enumerate(subjects):
        subj_df = df[df['subject_id'] == subj].sort_values('time')
        for _, row in subj_df.iterrows():
            ax.scatter(row['time'], i, c=[colours[row['event']]], s=100, zorder=3)
        # Connect events with line
        ax.plot(subj_df['time'], [i] * len(subj_df), 'k-', alpha=0.3, zorder=1)
    
    # Legend
    for event, colour in colours.items():
        ax.scatter([], [], c=[colour], label=event, s=100)
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Subject')
    ax.set_yticks(range(len(subjects)))
    ax.set_yticklabels(subjects)
    ax.legend(loc='upper right')
    ax.set_title('Individual Patient Journeys')
    
    return fig, ax

plot_journeys(df_covariate, n_subjects=15)
plt.show()

## Summary

The discretiser framework provides:

1. **Survival Models**: Weibull, Exponential, LogNormal, Gamma, and composites
2. **Point Masses**: For simultaneous events via `PointMassPlusContinuous`
3. **Event Triggering**: One event can trigger others with specified timing
4. **Subject Covariates**: Survival parameters can depend on subject features
5. **Competing Risks**: Multiple events compete; winner is minimum time
6. **Terminal Events**: Outcome and censoring end the simulation